# Step 5 - Plotting bicycle network analysis results
## Project: Algorithmic bicycle network design
#### Michael Szell, Tyler Perlman, Sayat Mimar, Gourab Ghoshal, Roberta Sinatra

This notebook takes the results from 03_poi_based_generation and 04_analyze_results, and plots them.

Contact: Michael Szell (michael.szell@gmail.com)  
Created: 2020-07-08  
Last modified: 2020-12-22

## Preliminaries

### Parameters

In [ ]:
debug = False # If True, will produce plots and/or verbose output to double-check
%run -i "../parameters/parameters.py"

### Setup

In [ ]:
%run -i path.py
%run -i setup.py

%load_ext watermark
%watermark -n -v -m -g -iv

### Functions

In [ ]:
%run -i functions.py

## Plot

Some plot functions love to plot inline. To suppress that and only export the plots, we use ``%%capture``.  
To check that plots are being generated while running the code below, go into the ``plots/placeid`` folder.

### Analysis plots

In [ ]:
%%capture
for placeid, placeinfo in cities.items():
    print(placeid + ": Plotting analysis results")
    
    # Load existing networks
    G_biketrack = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'biketrack')
    G_carall = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'carall')
    G_bikeable = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'bikeable')
    G_biketrack_onstreet = intersect_igraphs(G_biketrack, G_carall)
    G_bikeable_onstreet = intersect_igraphs(G_bikeable, G_carall)
    
    # PLOT Analysis
    filename = placeid + '_poi_' + poi_source + "_" + prune_measure + ".csv"
    analysis_result = np.genfromtxt(PATH["results"] + placeid + "/" + filename, delimiter=',', names=True)
    
    filename = placeid + '_poi_' + poi_source + "_mst.csv"
    analysis_mst_result = np.genfromtxt(PATH["results"] + placeid + "/" + filename, delimiter=',', names=True)
    
    filename = placeid + '_poi_' + poi_source + "_" + prune_measure + "_carminusbike.csv"
    analysis_carminusbike_result = np.genfromtxt(PATH["results"] + placeid + "/" + filename, delimiter=',', names=True)
    
    filename = placeid + '_poi_' + poi_source + "_" + prune_measure + "_carconstrictedbike.csv"
    analysis_carconstrictedbike_result = np.genfromtxt(PATH["results"] + placeid + "/" + filename, delimiter=',', names=True)
    
    filename = placeid + "_existing.csv"
    analysis_existing = np.genfromtxt(PATH["results"] + placeid + "/" + filename, delimiter=',', names=True)
    
    fig, axes = plt.subplots(nrows=2, ncols=9, figsize=(18, 5))
    # Bike network
    keys = ["length","coverage","directness_lcc","poi_coverage","components","overlap_biketrack","overlap_bikeable","efficiency_global","efficiency_local"]
    for i, ax in enumerate(axes[0]):
        key = keys[i]
        if i != 5 and i != 6:
            ax.plot(prune_quantiles, analysis_result[key], 'b', label='Grown')
            xmin, xmax = ax.get_xlim()
            ax.plot([xmin, xmax], [analysis_mst_result[key], analysis_mst_result[key]], ':', color='grey', label='MST')
        elif i == 5: # overlaps need to be made relative
            ax.plot(prune_quantiles, analysis_result[key] / analysis_result["length"], 'b', label='Grown')
        elif i == 6:
            ax.plot(prune_quantiles, analysis_result[key] / analysis_result["length"], 'b', label='Grown')
        try:
            if i < 5:
                ax.set_xticklabels([])
                ax.plot([xmin, xmax], [analysis_existing[key][1], analysis_existing[key][1]], '--b', label='Protected')
                ax.plot([xmin, xmax], [analysis_existing[key][0], analysis_existing[key][0]], ':b', label='Bikeable')
            else:
                ax.set_xlabel('Prune quantile')
                ax.set_xticks([0, 0.5, 1])
        except:
            pass
        
        
        ax.set_title(key)
        ax.set_xticks([0, 0.5, 1])
        if i == 0: 
            ax.set_ylabel('Bicycle network')
            ax.legend(loc='center left', bbox_to_anchor=(-1.5, 0.5))
        if i == len(axes[0])-1:
            ymin, ymax = ax.get_ylim()
            ax.text(xmax, ymin-(ymax-ymin)*0.6, placeid, fontsize=18, horizontalalignment='right')
            ax.text(xmax, ymin-(ymax-ymin), prune_measure, fontsize=14, horizontalalignment='right')
            ax.text(xmax, ymin-(ymax-ymin)*1.2, poi_source, fontsize=14, horizontalalignment='right')

    # Car network
    for i, ax in enumerate(axes[1]):
        key = keys[i]
        if i == 0: ax.set_ylabel('Car network')
        if i < 5:
            ax.plot(prune_quantiles, analysis_carminusbike_result[key], 'k')
            ax.set_xlabel('Prune quantile')
            ax.set_xticks([0, 0.5, 1])
        else:
            ax.set_visible(False) # No efficiencies
        if key == "directness_lcc":
            ax.plot(prune_quantiles, analysis_carconstrictedbike_result[key], 'r')
    fig.savefig(PATH["plots"] + placeid + "/" + placeid + '_analysis_poi_' + poi_source + "_" + prune_measure + '.png')

### Network plots

In [ ]:
%%capture
for placeid, placeinfo in cities.items():
    print(placeid + ": Plotting networks")
    
    # EXISTING INFRASTRUCTURE
    # Load networks
    G_biketrack = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'biketrack')
    G_carall = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'carall')
    G_biketrackcarall = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'biketrackcarall')
    G_bikeable = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'bikeable')
    map_center = nxdraw(G_carall, "carall")

    # Load POIs
    with open(PATH["data"] + placeid + "/" + placeid + '_poi_' + poi_source + '_nnidscarall.csv') as f:
        nnids = [int(line.rstrip()) for line in f]
    nodesize_poi = nodesize_from_pois(nnids)
    
    # PLOT existing networks
    fig = initplot()
    nxdraw(G_carall, "carall", map_center)
    nxdraw(G_carall, "poi_unreached", map_center, nnids, "nx.draw_networkx_nodes", nodesize_poi)
    plt.savefig(PATH["plots"] + placeid + "/" + placeid + '_carall_poi_' + poi_source + '.pdf', bbox_inches="tight")
    plt.savefig(PATH["plots"] + placeid + "/" + placeid + '_carall_poi_' + poi_source + '.png', bbox_inches="tight", dpi=plotparam["dpi"])
    plt.close()
    
    fig = initplot()
    nxdraw(G_carall, "carall", map_center)
    plt.savefig(PATH["plots"] + placeid + "/" + placeid + '_carall.pdf', bbox_inches="tight")
    plt.savefig(PATH["plots"] + placeid + "/" + placeid + '_carall.png', bbox_inches="tight", dpi=plotparam["dpi"])
    plt.close()
    
    fig = initplot()
    nxdraw(G_biketrack, "biketrack", map_center)
    plt.savefig(PATH["plots"] + placeid + "/" + placeid + '_biketrack.pdf', bbox_inches="tight")
    plt.savefig(PATH["plots"] + placeid + "/" + placeid + '_biketrack.png', bbox_inches="tight", dpi=plotparam["dpi"])
    plt.close()
    
    fig = initplot()
    nxdraw(G_bikeable, "bikeable", map_center)
    plt.savefig(PATH["plots"] + placeid + "/" + placeid + '_bikeable.pdf', bbox_inches="tight")
    plt.savefig(PATH["plots"] + placeid + "/" + placeid + '_bikeable.png', bbox_inches="tight", dpi=plotparam["dpi"])
    plt.close()
    
    fig = initplot()
    nxdraw(G_carall, "carall", map_center)
    nxdraw(G_biketrack, "biketrack", map_center, list(set([v["id"] for v in G_biketrack.vs]).intersection(set([v["id"] for v in G_carall.vs]))))
    nxdraw(G_biketrack, "biketrack_offstreet", map_center, list(set([v["id"] for v in G_biketrack.vs]).difference(set([v["id"] for v in G_carall.vs]))))
    plt.savefig(PATH["plots"] + placeid + "/" + placeid + '_biketrackcarall.pdf', bbox_inches="tight")
    plt.savefig(PATH["plots"] + placeid + "/" + placeid + '_biketrackcarall.png', bbox_inches="tight", dpi=plotparam["dpi"])
    plt.close()
    
    
    # GENERATED, POI BASED
    # Load results
    filename = placeid + '_poi_' + poi_source + "_" + prune_measure + ".pickle"
    with open(PATH["results"] + placeid + "/" + filename, 'rb') as f:
        res = pickle.load(f)
    if debug: pp.pprint(res)
        
    # PLOT abstract MST
    fig = initplot()
    nxdraw(G_carall, "carall", map_center)
    nxdraw(res["MST_abstract"], "abstract", map_center, weighted = 6)
    nxdraw(G_carall, "poi_unreached", map_center, nnids, "nx.draw_networkx_nodes", nodesize_poi)
    nxdraw(G_carall, "poi_reached", map_center, list(set([v["id"] for v in res["MST"].vs]).intersection(set(nnids))), "nx.draw_networkx_nodes", nodesize_poi)
    plt.savefig(PATH["plots"] + placeid + "/" + placeid + '_MSTabstract_poi_' + poi_source + '.pdf', bbox_inches="tight")
    plt.savefig(PATH["plots"] + placeid + "/" + placeid + '_MSTabstract_poi_' + poi_source + '.png', bbox_inches="tight", dpi=plotparam["dpi"])
    plt.close()
    
    # PLOT MST all together
    fig = initplot()
    nxdraw(G_carall, "carall")
    nxdraw(res["MST"], "bikegrown", map_center, nodesize = nodesize_grown)
    nxdraw(G_carall, "poi_unreached", map_center, nnids, "nx.draw_networkx_nodes", nodesize_poi)
    nxdraw(G_carall, "poi_reached", map_center, list(set([v["id"] for v in res["MST"].vs]).intersection(set(nnids))), "nx.draw_networkx_nodes", nodesize_poi)
    plt.savefig(PATH["plots"] + placeid + "/" + placeid + '_MSTall_poi_' + poi_source + '.pdf', bbox_inches="tight")
    plt.savefig(PATH["plots"] + placeid + "/" + placeid + '_MSTall_poi_' + poi_source + '.png', bbox_inches="tight", dpi=plotparam["dpi"])
    plt.close()
    
    # PLOT MST all together with abstract
    fig = initplot()
    nxdraw(G_carall, "carall", map_center)
    nxdraw(res["MST"], "bikegrown", map_center, nodesize = 0)
    nxdraw(res["MST_abstract"], "abstract", map_center, weighted = 6)
    nxdraw(G_carall, "poi_unreached", map_center, nnids, "nx.draw_networkx_nodes", nodesize_poi)
    nxdraw(G_carall, "poi_reached", map_center, list(set([v["id"] for v in res["MST"].vs]).intersection(set(nnids))), "nx.draw_networkx_nodes", nodesize_poi)
    plt.savefig(PATH["plots"] + placeid + "/" + placeid + '_MSTabstractall_poi_' + poi_source + '.pdf', bbox_inches="tight")
    plt.savefig(PATH["plots"] + placeid + "/" + placeid + '_MSTabstractall_poi_' + poi_source + '.png', bbox_inches="tight", dpi=plotparam["dpi"])
    plt.close()
    
    # PLOT abstract greedy triangulation (this can take some minutes)
    for GT_abstract, prune_quantile in zip(res["GT_abstracts"], res["prune_quantiles"]):
        fig = initplot()
        nxdraw(G_carall, "carall")
        nxdraw(GT_abstract, "abstract", map_center, drawfunc = "nx.draw_networkx_edges", nodesize = nodesize_poi, weighted = True)
        nxdraw(G_carall, "poi_unreached", map_center, nnids, "nx.draw_networkx_nodes", nodesize_poi)
        nxdraw(G_carall, "poi_reached", map_center, list(set([v["id"] for v in GT_abstract.vs]).intersection(set(nnids))), "nx.draw_networkx_nodes", nodesize_poi)
        plt.savefig(PATH["plots"] + placeid + "/" + placeid + '_GTabstract_poi_' + poi_source + "_" + prune_measures[prune_measure] + "{:.3f}".format(prune_quantile) + '.png', bbox_inches="tight", dpi=plotparam["dpi"])
        plt.close()
    
    # PLOT all together (this can take some minutes)
    for GT, prune_quantile in zip(res["GTs"], res["prune_quantiles"]):
        fig = initplot()
        nxdraw(G_carall, "carall")
        nxdraw(GT, "bikegrown", map_center, nodesize = nodesize_grown)
        nxdraw(G_carall, "poi_unreached", map_center, nnids, "nx.draw_networkx_nodes", nodesize_poi)
        nxdraw(G_carall, "poi_reached", map_center, list(set([v["id"] for v in GT.vs]).intersection(set(nnids))), "nx.draw_networkx_nodes", nodesize_poi)
        plt.savefig(PATH["plots"] + placeid + "/" + placeid + '_GTall_poi_' + poi_source + "_" + prune_measures[prune_measure] + "{:.3f}".format(prune_quantile) + '.png', bbox_inches="tight", dpi=plotparam["dpi"])
        plt.close()
        
    # PLOT all together with overlaps (this can take some minutes)
    for GT, prune_quantile in zip(res["GTs"], res["prune_quantiles"]):
        fig = initplot()
        nxdraw(G_carall, "carall")
        nxdraw(G_biketrack, "biketrack", map_center, list(set([v["id"] for v in G_biketrack.vs]).intersection(set([v["id"] for v in G_carall.vs]))))
        nxdraw(GT, "bikegrown", map_center, nodesize = 0)
        nxdraw(GT, "highlight", map_center, list(set([v["id"] for v in G_biketrack.vs]).intersection(set([v["id"] for v in GT.vs]))))
        nxdraw(G_carall, "poi_unreached", map_center, nnids, "nx.draw_networkx_nodes", nodesize_poi)
        nxdraw(G_carall, "poi_reached", map_center, list(set([v["id"] for v in GT.vs]).intersection(set(nnids))), "nx.draw_networkx_nodes", nodesize_poi)
        plt.savefig(PATH["plots"] + placeid + "/" + placeid + '_GTalloverlapbiketrack_poi_' + poi_source + "_" + prune_measures[prune_measure] + "{:.3f}".format(prune_quantile) + '.png', bbox_inches="tight", dpi=plotparam["dpi"])
        plt.close()
        
    for GT, prune_quantile in zip(res["GTs"], res["prune_quantiles"]):
        fig = initplot()
        nxdraw(G_carall, "carall")
        nxdraw(G_bikeable, "bikeable", map_center, list(set([v["id"] for v in G_bikeable.vs]).intersection(set([v["id"] for v in G_carall.vs]))))
        nxdraw(GT, "bikegrown", map_center, nodesize = 0)
        nxdraw(GT, "highlight", map_center, list(set([v["id"] for v in G_bikeable.vs]).intersection(set([v["id"] for v in GT.vs]))))
        nxdraw(G_carall, "poi_unreached", map_center, nnids, "nx.draw_networkx_nodes", nodesize_poi)
        nxdraw(G_carall, "poi_reached", map_center, list(set([v["id"] for v in GT.vs]).intersection(set(nnids))), "nx.draw_networkx_nodes", nodesize_poi)
        plt.savefig(PATH["plots"] + placeid + "/" + placeid + '_GTalloverlapbikeable_poi_' + poi_source + "_" + prune_measures[prune_measure] + "{:.3f}".format(prune_quantile) + '.png', bbox_inches="tight", dpi=plotparam["dpi"])
        plt.close()

#### Cover plots

In [ ]:
%%capture
for placeid, placeinfo in cities.items():
    print(placeid + ": Plotting network covers")

    # Load networks
    G_biketrack = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'biketrack')
    G_carall = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'carall')
    G_biketrackcarall = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'biketrackcarall')
    G_bikeable = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'bikeable')
    map_center = nxdraw(G_carall, "carall")
    
    # Load POIs
    with open(PATH["data"] + placeid + "/" + placeid + '_poi_' + poi_source + '_nnidscarall.csv') as f:
        nnids = [int(line.rstrip()) for line in f]
    nodesize_poi = nodesize_from_pois(nnids)
    
    # Load results
    filename = placeid + '_poi_' + poi_source + "_" + prune_measure + ".pickle"
    with open(PATH["results"] + placeid + "/" + filename, 'rb') as f:
        res = pickle.load(f)
    
    # Load covers
    filename = placeid + '_poi_' + poi_source + "_" + prune_measure + "_covers"
    with open(PATH["results"] + placeid + "/" + filename + ".pickle",'rb') as f:
        covs = pickle.load(f)
    filename = placeid + "_"  + "existing_covers"
    with open(PATH["results"] + placeid + "/" + filename + ".pickle",'rb') as f:
        cov_car = pickle.load(f)['carall']
    
    # Construct and plot patches from covers
    patchlist_car, patchlist_car_holes = cov_to_patchlist(cov_car, map_center)
    for GT, prune_quantile, cov in zip(res["GTs"], res["prune_quantiles"], covs.values()):
        fig = initplot()
        
        # Covers
        axes = fig.add_axes([0, 0, 1, 1]) # left, bottom, width, height (range 0 to 1)
        patchlist_bike, patchlist_bike_holes = cov_to_patchlist(cov, map_center)
        
        # We have this contrived order due to alphas, holes, and matplotlib's inability to draw polygon patches with holes. This only works because the car network is a superset of the bike network.
        # car orange, bike white, bike blue, bike holes white, bike holes orange, car holes white
        patchlist_combined = patchlist_car + patchlist_bike + patchlist_bike + patchlist_bike_holes+ patchlist_bike_holes + patchlist_car_holes
        pc = PatchCollection(patchlist_combined)
        colors = np.array([[255/255,115/255,56/255,0.2] for _ in range(len(patchlist_car))]) # car orange
        colors = np.append(colors, [[1,1,1,1] for _ in range(len(patchlist_bike))], axis = 0) # bike white
        colors = np.append(colors, [[0,142/255,204/255,0.4] for _ in range(len(patchlist_bike))], axis = 0) # bike blue
        if len(patchlist_bike_holes):
            colors = np.append(colors, [[1,1,1,1] for _ in range(len(patchlist_bike_holes))], axis = 0) # bike holes white
        if len(patchlist_bike_holes):
            colors = np.append(colors, [[255/255,115/255,56/255,0.2] for _ in range(len(patchlist_bike_holes))], axis = 0) # bike holes orange
        if len(patchlist_car_holes):
            colors = np.append(colors, [[1,1,1,1] for _ in range(len(patchlist_car_holes))], axis = 0) # car holes white
        pc.set_facecolors(colors)
        pc.set_edgecolors(np.array([[0,0,0,0.4] for _ in range(len(patchlist_combined))])) # remove this line if the outline of the full coverage should remain
        axes.add_collection(pc)
        axes.set_aspect('equal')
        axes.set_xmargin(0.01)
        axes.set_ymargin(0.01)
        axes.plot()
        
        # Networks
        nxdraw(G_carall, "carall", map_center)
        nxdraw(GT, "bikegrown", map_center, nodesize = nodesize_grown)
        nxdraw(G_carall, "poi_unreached", map_center, nnids, "nx.draw_networkx_nodes", nodesize_poi)
        nxdraw(G_carall, "poi_reached", map_center, list(set([v["id"] for v in GT.vs]).intersection(set(nnids))), "nx.draw_networkx_nodes", nodesize_poi)
        plt.savefig(PATH["plots"] + placeid + "/" + placeid + '_GTallcover_poi_' + poi_source + "_" + prune_measures[prune_measure] + "{:.3f}".format(prune_quantile) + '.png', bbox_inches="tight", dpi=plotparam["dpi"])

In [ ]:
Audio(sound_file, autoplay=True)